# Mamdani Fuzzy Inference System

**Classic Tipping Problem**

In [ ]:
!pip install pyfuzzy-toolbox -q

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from fuzzy_systems.core import LinguisticVariable
from fuzzy_systems.inference import MamdaniSystem

plt.style.use('seaborn-v0_8-darkgrid')
%matplotlib inline

### 1. Create Mamdani System

In [ ]:
# Create Mamdani system
fis = MamdaniSystem()

# Add input variables with universe
service = fis.add_input('service', (0, 10))
food = fis.add_input('food', (0, 10))

# Add output variable
tip = fis.add_output('tip', (0, 30))

print(f"System created with {len(fis.input_variables)} inputs and {len(fis.output_variables)} outputs")

### 2. Define Membership Functions

In [ ]:
# Service quality terms
fis.add_term('service', 'poor', 'triangular', (0, 0, 5))
fis.add_term('service', 'good', 'triangular', (0, 5, 10))
fis.add_term('service', 'excellent', 'triangular', (5, 10, 10))

# Food quality terms
fis.add_term('food', 'bad', 'triangular', (0, 0, 5))
fis.add_term('food', 'good', 'triangular', (0, 5, 10))
fis.add_term('food', 'delicious', 'triangular', (5, 10, 10))

# Tip percentage terms
fis.add_term('tip', 'low', 'triangular', (0, 0, 15))
fis.add_term('tip', 'medium', 'triangular', (0, 15, 30))
fis.add_term('tip', 'high', 'triangular', (15, 30, 30))

# Visualize
fis.plot_variables(figsize=(14, 4))

### 3. Define Fuzzy Rules

In [ ]:
# Add fuzzy rules
fis.add_rule("IF service IS poor OR food IS bad THEN tip IS low")
fis.add_rule("IF service IS good THEN tip IS medium")
fis.add_rule("IF service IS excellent OR food IS delicious THEN tip IS high")

fis.print_rules()

### 4. Single Inference Example

In [ ]:
# Evaluate with specific values
result = fis.evaluate(service=7.5, food=8.0)

print(f"Input: service=7.5, food=8.0")
print(f"Output: tip={result['tip']:.2f}%")

### 5. Detailed Inference Information

In [ ]:
# Get detailed inference process
details = fis.evaluate_detailed(service=7.5, food=8.0)

print("Fuzzified inputs:")
for var, terms in details['fuzzified_inputs'].items():
    print(f"  {var}: {terms}")

print("\nActivated rules:")
for rule_info in details['activated_rules']:
    print(f"  Rule {rule_info['rule_index']}: {rule_info['rule']}")
    print(f"    Firing strength: {rule_info['firing_strength']:.3f}")

print(f"\nFinal output: {details['outputs']}")

### 6. Surface Plot: Response Over Input Space

In [ ]:
# Generate surface plot
service_range = np.linspace(0, 10, 30)
food_range = np.linspace(0, 10, 30)
tip_surface = np.zeros((len(food_range), len(service_range)))

for i, f in enumerate(food_range):
    for j, s in enumerate(service_range):
        result = fis.evaluate(service=s, food=f)
        tip_surface[i, j] = result['tip']

# 3D surface plot
from mpl_toolkits.mplot3d import Axes3D
S, F = np.meshgrid(service_range, food_range)

fig = plt.figure(figsize=(12, 8))
ax = fig.add_subplot(111, projection='3d')
surf = ax.plot_surface(S, F, tip_surface, cmap='viridis', alpha=0.8)
ax.set_xlabel('Service Quality')
ax.set_ylabel('Food Quality')
ax.set_zlabel('Tip (%)')
ax.set_title('Mamdani FIS: Tipping Response Surface')
plt.colorbar(surf, shrink=0.5)
plt.show()

### 7. Contour Plot

In [ ]:
# 2D contour plot
plt.figure(figsize=(10, 8))
contour = plt.contourf(S, F, tip_surface, levels=15, cmap='viridis')
plt.colorbar(contour, label='Tip (%)')
plt.xlabel('Service Quality')
plt.ylabel('Food Quality')
plt.title('Tipping Decision Contour Map')
plt.grid(True, alpha=0.3)
plt.show()

### 🎯 Exercise: Test Different Scenarios

In [ ]:
# Test multiple scenarios
scenarios = [
    (2, 3, 'Poor service, bad food'),
    (5, 5, 'Average everything'),
    (8, 9, 'Great experience'),
    (3, 9, 'Poor service, great food'),
    (9, 3, 'Great service, bad food')
]

for s, f, desc in scenarios:
    result = fis.evaluate(service=s, food=f)
    print(f"{desc:30s} → Tip: {result['tip']:5.2f}%")